<a href="https://colab.research.google.com/github/Cecax27/DS-Stardew-Valley-Crops-Profit/blob/main/notebooks/Stardew_valley_web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Web Scraper on Stardew Valley wiki**

Im going to do web scraping with Python on Stardew Valley Wiki to extract information about crops, and then make an analysis over the recolected data

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
home_url = 'https://stardewvalleywiki.com'

In [ ]:
url = 'https://stardewvalleywiki.com/Crops'

*Making the request.*

In [ ]:
wiki = requests.get(url)

## Parse HTML

In [ ]:
s = BeautifulSoup(wiki.text, 'lxml')

## Finding the urls

In [ ]:
crops_names = s.find('div', attrs = {'class' : 'mw-parser-output'}).find_all('h3')[10:-1]

Making a list with all the links

In [ ]:
crops_link_list = [home_url + crop_name.find_all('a')[1].get('href') for crop_name in crops_names]

In [ ]:
crops_link_list[0:5]

['https://stardewvalleywiki.com/Blue_Jazz',
 'https://stardewvalleywiki.com/Cauliflower',
 'https://stardewvalleywiki.com/Coffee_Bean',
 'https://stardewvalleywiki.com/Garlic',
 'https://stardewvalleywiki.com/Green_Bean']

In [ ]:
crops_names_list = [crop_name.get_text().strip() for crop_name in crops_names]

Making all the requests

In [ ]:
crops_soup = [BeautifulSoup(requests.get(link).text, 'lxml') for link in crops_link_list]

Function to extract the data from each request

In [ ]:
# Function to extract information about the seed

def get_seed_information(link):
  try:
    s = requests.get(link)
  except Exception as e:
    return e
  raw_information = BeautifulSoup(s.text, 'lxml')
  price_dict = dict()
  for row in raw_information.find('table').find_all('tr'):
    title = row.find('td').get_text().strip().replace(':', '').lower().replace(' ', '_')
    if title == 'general_store':
      try:
        price_dict[title+'_price'] = int(row.find_all('td')[1].find_all('span')[1].get_text()[:-1])
      except:
        price_dict[title+'_price'] = np.nan
    elif title == 'jojamart':
      try:
        price_dict[title+'_price'] = int(row.find_all('td')[1].find_all('span')[1].get_text()[:-1])
      except:
        price_dict[title+'_price'] = np.nan
    elif title == 'oasis':
      price_dict[title+'_price'] = int(row.find_all('td')[1].find_all('span')[1].get_text()[:-1])
    elif title == 'traveling_cart':
      price_dict[title+'_price_min'] = int(row.find_all('td')[1].get_text().split('"')[-1].split('g')[0].replace(',','').split('–')[0])
      price_dict[title+'_price_max'] = int(row.find_all('td')[1].get_text().split('"')[-1].split('g')[0].replace(',','').split('–')[1])
    elif title == 'night_market(winter_15)':
      price_dict[title.split('(')[0]+'_price'] = int(row.find_all('td')[1].find_all('span')[1].get_text()[:-1])
  return price_dict

In [ ]:
# Testing
print(get_seed_information('https://stardewvalleywiki.com/Starfruit_Seeds'))

{'general_store_price': nan, 'jojamart_price': nan, 'oasis_price': 400, 'traveling_cart_price_min': 600, 'traveling_cart_price_max': 1000}


In [ ]:
def get_crop(raw_information):
  prices_sufix = ['regular', 'silver', 'gold', 'iridium']
  inf_dict = {
      'name' : raw_information[0].get_text().strip(),
      'description' : raw_information[2].get_text().strip()
  }

  for data in raw_information[4:]:
    field = data.find('td').get_text().strip().lower().replace(' ', '_')
    if '\t' in field or ':' in field or '/' in field or field == '' or field[0].isdigit():
      continue
    if field == 'sell_price' or field == 'sell_prices':
      try:
        prices = data.find('table').find_all('td')
        prices = list(map(lambda i: i.get_text().split('g')[0], prices))[1:]
        prices = list(filter(lambda i: i != '', prices))
        prices = list(map(lambda i: int(i), prices))
        for index, price in enumerate(prices):
          inf_dict[field+'_'+prices_sufix[index]] = price
        continue
      except Exception as e:
        try:
          index = raw_information.index(data) + 2
          prices = raw_information[index].find('table').find_all('td')
          prices = list(map(lambda i: i.get_text().split('g')[0].replace(',',''), prices))[1:]
          prices = list(filter(lambda i: i != '', prices))
          prices = list(map(lambda i: int(i), prices))
          for index, price in enumerate(prices):
            inf_dict[field+'_'+prices_sufix[index]] = price
          continue
        except Exception as e:
          print(e)
        continue
    if field == 'seed':
      try:
        url = home_url + data.find_all('td')[1].find('a').get('href')
        inf_dict.update(get_seed_information(url))
      except Exception as e:
        pass
    try:
      inf_dict[field] = data.find_all('td')[1].get_text().strip().replace(' • ', ',')
    except:
      pass

  return inf_dict

In [ ]:
# Testing

get_crop(crops_soup[0].find('table').find_all('tr'))

{'name': 'Blue Jazz',
 'description': 'The flower grows in a sphere to invite as many butterflies as possible.',
 'general_store_price': 30,
 'jojamart_price': 37,
 'traveling_cart_price_min': 100,
 'traveling_cart_price_max': 1000,
 'night_market_price': 30,
 'seed': 'Jazz Seeds',
 'growth_time': '7 days',
 'season': 'Spring',
 'xp': '10 Farming XP',
 'sell_prices_regular': 50,
 'sell_prices_silver': 62,
 'sell_prices_gold': 75,
 'sell_prices_iridium': 100,
 'base': 'Artisan (+40%)'}

Extracting all the data

In [ ]:
df_data = [get_crop(crop.find('table').find_all('tr')) for crop in crops_soup]
columns = [list(item.keys()) for item in df_data]
columns = set()
for sublist in [list(item.keys()) for item in df_data]:
  for element in sublist:
    columns.add(element)
columns = list(columns)
df = pd.DataFrame(df_data, columns = columns)

list index out of range
'NoneType' object has no attribute 'find_all'


In [ ]:
name_column = df.pop('name')
df.insert(0, 'name', name_column)
df.sort_values('name')
#Joining the sell_price columns

df['sell_price_regular'] = df['sell_price_regular'].combine_first(df['sell_prices_regular'])
df = df.drop('sell_prices_regular', axis=1)

df['sell_price_silver'] = df['sell_price_silver'].combine_first(df['sell_prices_silver'])
df = df.drop('sell_prices_silver', axis=1)

df['sell_price_gold'] = df['sell_price_gold'].combine_first(df['sell_prices_gold'])
df = df.drop('sell_prices_gold', axis=1)

df['sell_price_iridium'] = df['sell_price_iridium'].combine_first(df['sell_prices_iridium'])
df = df.drop('sell_prices_iridium', axis=1)

# Create separate columns for each season
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
for season in seasons:
    df[season.lower()] = df['season'].str.contains(season, case=False)

# Fill the values with True or False
df = df.replace({True: 'True', False: 'False'})
df = df.drop('season', axis=1)

# Print the resulting dataframe
df.head()

,name,sell_price_gold,growth_time,traveling_cart_price_max,seed,energy,base,night_market_price,source,sell_price_regular,...,general_store_price,sell_price_silver,jojamart_price,artisan_sell_price,description,traveling_cart_price_min,spring,summer,fall,winter
0,Blue Jazz,75.0,7 days,1000.0,Jazz Seeds,NaN,Artisan (+40%),30.0,NaN,50.0,...,30.0,62.0,37.0,NaN,The flower grows in a sphere to invite as many...,100.0,True,False,False,False
1,Cauliflower,262.0,12 days,1000.0,Cauliflower Seeds,NaN,Artisan (+40%),80.0,NaN,175.0,...,80.0,218.0,100.0,NaN,"Valuable, but slow-growing. Despite its pale c...",120.0,True,False,False,False
2,Coffee Bean,22.0,10 days,NaN,Coffee Bean,Inedible,NaN,NaN,"Dust Sprite,Traveling Cart",15.0,...,NaN,18.0,NaN,150g,Plant in spring or summer to grow a coffee pla...,NaN,True,True,False,False
3,Garlic,90.0,4 days,1000.0,Garlic Seeds,NaN,Artisan (+40%),40.0,NaN,60.0,...,40.0,75.0,NaN,NaN,Adds a wonderful zestiness to dishes. High qua...,100.0,True,False,False,False
4,Green Bean,60.0,10 days,1000.0,Bean Starter,NaN,Artisan (+40%),60.0,NaN,40.0,...,60.0,50.0,75.0,NaN,"A juicy little bean with a cool, crisp snap.",100.0,True,False,False,False


In [ ]:
df['growth_time'] = (
    df
    .growth_time
    .fillna('0')
    .apply(lambda i: i.split(' ')[0])
    .astype(int)
)

In [ ]:
df.to_csv('crops.csv')

**Deleting the Ginger Island crops**

I deleted the Ginger Island crops because you can't buy the seed in the stores. You can trade they but it's more difficult to compare the price of the trade with gold. The Ginger Island crops are:

* Taro roots
* Pineapple

Also, I deleted some crops in similar situations, like:

* Tea Leaves
* Sweet Gem Berry
* Mixed Seeds
* Fiber
* Cactus Fruit
* Strawberry
* Coffee Bean




In [ ]:
df = df.drop(df[df['name'] == 'Taro Root'].index)
df = df.drop(df[df['name'] == 'Pineapple'].index)

df = df.drop(df[df['name'] == 'Tea Leaves'].index)
df = df.drop(df[df['name'] == 'Sweet Gem Berry'].index)
df = df.drop(df[df['name'] == 'Mixed Seeds'].index)
df = df.drop(df[df['name'] == 'Fiber'].index)
df = df.drop(df[df['name'] == 'Cactus Fruit'].index)
df = df.drop(df[df['name'] == 'Strawberry'].index)
df = df.drop(df[df['name'] == 'Coffee Bean'].index)

In [ ]:
df.columns

Index(['name', 'sell_price_gold', 'growth_time', 'traveling_cart_price_max',
       'seed', 'energy', 'base', 'night_market_price', 'source',
       'sell_price_regular', 'sell_price_iridium', 'oasis_price', 'xp',
       'general_store_price', 'sell_price_silver', 'jojamart_price',
       'artisan_sell_price', 'description', 'traveling_cart_price_min',
       'spring', 'summer', 'fall', 'winter', 'price_min', 'gold_per_day'],
      dtype='object')

In [ ]:
df['price_min'] = df[['night_market_price', 'oasis_price', 'general_store_price', 'jojamart_price', 'traveling_cart_price_min']].min(axis=1)

In [ ]:
df[['name', 'sell_price_regular', 'growth_time', 'price_min']].head()

,name,sell_price_regular,growth_time,price_min
0,Blue Jazz,50.0,7,30.0
1,Cauliflower,175.0,12,80.0
3,Garlic,60.0,4,40.0
4,Green Bean,40.0,10,60.0
5,Kale,110.0,6,70.0


In [ ]:
df['gold_per_day'] = ( df['sell_price_iridium'] - df['price_min'])  / df['growth_time']

In [ ]:
df[df['spring'] == 'True'][['name', 'sell_price_iridium', 'growth_time', 'price_min', 'gold_per_day']].sort_values('gold_per_day', ascending = False)

,name,sell_price_iridium,growth_time,price_min,gold_per_day
35,Ancient Fruit,1100.0,28,100.0,35.714286
8,Rhubarb,440.0,13,100.0,26.153846
5,Kale,220.0,6,70.0,25.000000
1,Cauliflower,350.0,12,80.0,22.500000
3,Garlic,120.0,4,40.0,20.000000
7,Potato,160.0,6,50.0,18.333333
6,Parsnip,70.0,4,20.0,12.500000
0,Blue Jazz,100.0,7,30.0,10.000000
10,Tulip,60.0,6,20.0,6.666667
11,Unmilled Rice,60.0,8,40.0,2.500000
